In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

In [ ]:
data_apart=pd.read_csv('../../Dataset_Apart/Cleaned/DatasetAirbnb_Cleaned_v1.csv')
data_apart.head()

In [ ]:
data_apart.info()

# **Comprobamos valores nulos**
---

In [ ]:
data_apart.isna().sum()

# **Descripcion de las variables numéricas**
---

In [ ]:
data_apart.describe()

# **Descripcion de las variables categóricas**
---

In [ ]:
data_apart.describe(include='O')

# **Conversion de localización a latitud, longitud**
---

In [ ]:
from time import sleep
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd

def lat_log(l):
    geolocator = Nominatim(user_agent="España")
    try:
        location = geolocator.geocode(l)
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    except:
        return 10000,10000
data_apart['Latitud_Longitud']=data_apart['Localizacion'].apply(lambda x: lat_log(x))



In [ ]:
data_apart['Latitud']=data_apart['Latitud_Longitud'].apply(lambda x: x[0])
data_apart['Longitud']=data_apart['Latitud_Longitud'].apply(lambda x: x[1])
data_apart.drop('Latitud_Longitud',axis=1,inplace=True)

In [ ]:
data_apart.drop('Localizacion',axis=1,inplace=True)

In [ ]:
data_apart.to_csv('../../Dataset_Apart/Cleaned/DatasetAirbnb_Cleaned_v1.csv',index=False)

In [ ]:
data_apart.head()

# **Estudio del Precio**
---

In [ ]:
import pandas as pd
import plotly.express as px

# Datos con latitud/longitud y valores
datos=data_apart[(data_apart['Latitud']!=10000)&(data_apart['Longitud']!=10000)][['Latitud','Longitud','Precio']]
fig = px.density_mapbox(datos, lat = 'Latitud', lon = 'Longitud', z = 'Precio',
                        radius = 7,
                        center = dict(lat = 40.45, lon = -3.6),
                        zoom = 4,
                        mapbox_style = 'open-street-map',
                        color_continuous_scale = 'rainbow',
                        opacity = 0.5)
fig.show() 


En el mapa se observan 3 focos principales, Madrid, Alicante y Navarra, como los lugares mas caros.

#### **Precio medio por tipo de apartamento**

In [ ]:
sns.barplot(data=data_apart.groupby('Tipo').agg({
    'Precio':'mean'
}).reset_index(), x='Tipo',y='Precio')


En principio el tipo de apartamento mas caro, son los de tipo en el campo, seguramente debido al mayor espacio que tienen sus propiedades de media.

#### **Relacion entre media de valoraciones y precio**

In [ ]:
sns.regplot(data=data_apart,x='Precio',y='Calidad')

#### **Relacion entre limpieza y precio**

In [ ]:
sns.regplot(data=data_apart,x='Precio',y='Limpieza')

In [ ]:
sns.

# **Clustering apartamentos**

In [ ]:
data_apart.columns

In [ ]:
columnas_innecesarias_cluster=['ID','Titulo','Descripcion Simple','URL','url_img','Localizacion']
data_cluster=data_apart.drop(columnas_innecesarias_cluster,axis=1)


In [ ]:
data_cluster

In [ ]:
imputer = SimpleImputer(strategy='mean')
imputer_cat = SimpleImputer(strategy='most_frequent')
cols_cat=['Wifi','Mascotas','Piscina','Parking']
not_cat=[col for col in data_cluster.columns if col not in cols_cat and col not in columnas_innecesarias_cluster]
data_cluster[cols_cat]=imputer_cat.fit_transform(data_cluster[cols_cat])
data_cluster[not_cat]=imputer.fit_transform(data_cluster[not_cat])

In [ ]:
data_apart.to_csv('../../Dataset_Apart/Cleaned/DatasetAirbnb_Cleaned_v1.csv',index=False)

In [ ]:
imputer = SimpleImputer(strategy='mean')
imputer_cat = SimpleImputer(strategy='most_frequent')
cols_cat=['Wifi','Mascotas','Piscina','Parking']
not_cat=[col for col in data_cluster.columns if col not in cols_cat]
data_cluster[cols_cat]=imputer_cat.fit_transform(data_cluster[cols_cat])
data_cluster[not_cat]=imputer.fit_transform(data_cluster[not_cat])

In [ ]:
data_cluster.isna().sum()

## **Método del codo**

In [ ]:
from sklearn.cluster import KMeans
Coste = []
krange = np.arange(2, 11) #krange = 2,3,4,5,6,7,8,9,10
# bucle para el cálculo de la función de coste (SSE) desde k= 2 hasta k= 10
for num in krange:
    kmeans = KMeans(n_clusters=num, n_init='auto', random_state=10, max_iter=100).fit(data_cluster)
    print(
        "Para k =",
        num,
        ', el coste (SSE)=',
        kmeans.inertia_,
    )
    Coste.append(kmeans.inertia_)

fig, ax = plt.subplots()
ax.set_title('Método del codo', fontsize = 16)
ax.set_ylabel("Coste (SSE)",fontsize=14)
ax.set_xlabel("Número de clústeres (k)", fontsize=14)
ax.plot(krange,Coste)
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score

for k in [3,4,5,6,7,8]:
    kmeans = KMeans(n_clusters=k, n_init= 'auto', random_state=10, max_iter=3000)
    Y_pred=kmeans.fit_predict(data_cluster) # Vector de asignación de etiquetas predichas para cada elemento
    data_cluster['id_cluster']=kmeans.labels_
    silhouette_avg = silhouette_score(data_cluster.drop('id_cluster',axis=1),data_cluster['id_cluster'])
    cal=calinski_harabasz_score(data_cluster.drop('id_cluster',axis=1),data_cluster['id_cluster'])
    print('\nPara un Nº de clusters: ',k ,
          '\nS: ',silhouette_avg,
          '\nCH: ',cal)
    print('-'*50)

El número óptimo de clusters son 3.

In [ ]:
k_val=8
kmeans = KMeans(n_clusters=k_val, n_init= 'auto', random_state=10, max_iter=3000)
Y_pred=kmeans.fit_predict(data_cluster) # Vector de asignación de etiquetas predichas para cada elemento
kmeans.labels_
data_cluster['id_cluster']=kmeans.labels_
#Boxplot con el comportamiento de los clusters para cada variable (Facilitar la caracterización)
for col in data_cluster.columns:
    sns.boxplot(data_cluster,x=col,hue='id_cluster')
    plt.show()

In [ ]:
data_cluster['id_cluster'].value_counts()

## **Clustering jerárquico**
---

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

def hierarchical_clustering(X, num_clusters):
    Z = linkage(X, method='ward')
    clusters = fcluster(Z, t=num_clusters, criterion='maxclust')
    return clusters
data_cluster.drop('id_cluster',axis=1,inplace=True)
num_clusters = 3
clusters = hierarchical_clustering(data_cluster, num_clusters=num_clusters)

# Visualización del dendrograma
plt.figure()
dendrogram(linkage(data_cluster, method='ward'))
plt.show()


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(data_cluster)

# Visualización de los datos originales y los datos transformados
plt.figure(figsize=(8, 4))

# Datos transformados por PCA
plt.subplot(1, 2, 2)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c='red', edgecolor='k', s=50)
plt.title("Datos transformados por PCA")
plt.xlabel("Componente principal 1")
plt.ylabel("Componente principal 2")

plt.tight_layout()
plt.show()

In [ ]:
plt.subplot(1, 2, 2)
plt.scatter(X_pca[:, 0], X_pca[:, 2], c='red', edgecolor='k', s=50)
plt.title("Datos transformados por PCA")
plt.xlabel("Componente principal 1")
plt.ylabel("Componente principal 3")

plt.tight_layout()
plt.show()

In [ ]:
plt.subplot(1, 2, 2)
plt.scatter(X_pca[:, 1], X_pca[:, 2], c='red', edgecolor='k', s=50)
plt.title("Datos transformados por PCA")
plt.xlabel("Componente principal 2")
plt.ylabel("Componente principal 3")

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
data_pca=pd.DataFrame(X_pca,columns=['PCA1','PCA2','PCA3'])
for k in [2,3,4,5]:
    kmeans = KMeans(n_clusters=k, n_init= 'auto', random_state=10, max_iter=3000)
    Y_pred=kmeans.fit_predict(data_pca) # Vector de asignación de etiquetas predichas para cada elemento
    data_pca['id_cluster']=kmeans.labels_
    silhouette_avg = silhouette_score(data_pca.drop('id_cluster',axis=1),data_pca['id_cluster'])
    cal=calinski_harabasz_score(data_pca.drop('id_cluster',axis=1),data_pca['id_cluster'])
    print('\nPara un Nº de clusters: ',k ,
          '\nS: ',silhouette_avg,
          '\nCH: ',cal)
    print('-'*50)

In [ ]:
k_val=3
kmeans = KMeans(n_clusters=k_val, n_init= 'auto', random_state=10, max_iter=3000)
Y_pred=kmeans.fit_predict(data_pca) # Vector de asignación de etiquetas predichas para cada elemento
kmeans.labels_
data_pca['id_cluster']=kmeans.labels_
#Boxplot con el comportamiento de los clusters para cada variable (Facilitar la caracterización)
for col in data_pca.columns:
    sns.boxplot(data_pca,x=col,hue='id_cluster')
    plt.show()

In [ ]:
from sklearn.cluster import DBSCAN

# Crear el modelo DBSCAN
dbscan = DBSCAN(eps=0.1, min_samples=20)

# Ajustar el modelo a los datos
dbscan.fit(data_cluster)

# Obtener las etiquetas de los clusters asignados a cada muestra
labels = dbscan.labels_

# Obtener el número de clusters encontrados (-1 indica muestras ruido)
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print("Número de clusters encontrados:", n_clusters)

# Obtener las muestras asignadas a cada cluster
clusters = {}
for i, label in enumerate(labels):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(i)

# Imprimir las muestras asignadas a cada cluster
for label, samples in clusters.items():
    print("Cluster", label)
    print("Muestras:", samples)
    print()